# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'Data/'

In [3]:
input_data = pd.read_sas(path+'LLCP2022.xpt')
input_data.shape

(445132, 328)

# BFRSS 2022

In [4]:
column_names = {
                '_SEX'    : 'sex',
                '_AGE_G'  :'age_group',
                '_EDUCAG' :'education',
                '_HLTHPLN':'has_insurance',
                '_INCOMG1':'income_group',
                '_RACE1'  :'race',
                'MARITAL' :'marital_status',
                'GENHLTH' :'general_health',
                'EXERANY2':'exercise_lately',
                'SLEPTIM1':'sleep_time',
                '_SMOKER3':'smoking',
                '_DRNKWK2':'alcohol_weekly',
                '_RFDRHV8':'heavy_drinker',
                'HTM4'    :'height',
                'WTKG3'   :'weight',
                '_BMI5'   :'bmi',
                '_BMI5CAT':'bmi_groups',
                'PERSDOC3':'has_doctor',
                'MEDCOST1':'affords_doctor',
                'CHECKUP1':'last_checkup',
                '_MICHD'  :'heart_problem',
                '_ASTHMS1':'asthma_history',
                'CVDSTRK3':'stroke',
                'ADDEPEV3':'depression',
                #######################
                'PDIABTS1':'high_bp',
                'PREDIAB2':'prediabetes',
                'DIABTYPE':'diabetes_type',
                'INSULIN1':'currently_insulin',
                'CHKHEMO3':'Hemoglobin',
                'EYEEXAM1':'dilated_pupils',
                'DIABEYE1':'eye_photo',
                'DIABEDU1':'diabetes_education',
                'FEETSORE':'sore_feet',
                'DIABETE4':'diabetes',
                }

In [5]:
column_values = {
                 column_names['_SEX']: {1:0, 2:1},
                 column_names['_AGE_G']: {1:0, 2:1, 3:2, 4:3, 5:4, 6:5},
                 column_names['_EDUCAG']: {1:0, 2:1, 3:2, 4:3, 9:np.nan},
                 column_names['_HLTHPLN']: {1:1, 2:0, 9: np.nan},
                 column_names['_INCOMG1']: {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 9:np.nan},
                 column_names['_RACE1'] : {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:np.nan},
                 column_names['MARITAL']: {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 9:np.nan},
                 column_names['GENHLTH']: {1:0, 2:1, 9:np.nan},
                 column_names['EXERANY2']: {1:1, 2:0, 7:np.nan, 9:np.nan},
                 column_names['SLEPTIM1']: {77:np.nan, 99:np.nan},
                 column_names['_SMOKER3']: {1:0, 2:1, 3:2, 4:3, 9:np.nan},
                 column_names['_DRNKWK2']:{99900:np.nan},
                 column_names['_RFDRHV8']: {1:0, 2:1, 9:np.nan},
                 column_names['_BMI5CAT']: {1:0, 2:1, 3:2, 4:3},
                 column_names['PERSDOC3']:{1:0, 2:1, 3:2, 7:np.nan, 9:np.nan},
                 column_names['MEDCOST1']:{1:1, 2:0, 7:np.nan, 9:np.nan},
                 column_names['CHECKUP1']: {1:0, 2:1, 3:2, 4:3, 7:np.nan, 8:4, 9:np.nan},
                 column_names['_MICHD']: {1:1, 2:0},
                 column_names['_ASTHMS1']: {3:0, 9:np.nan},
                 column_names['CVDSTRK3']:{1:1, 2:0, 7:np.nan, 9:np.nan},
                 column_names['ADDEPEV3']:{1:1, 2:0, 7:np.nan, 9:np.nan},
                 column_names['DIABETE4']: {1:2, 2:2, 3:0, 4:1, 7:np.nan, 9:np.nan},
                 column_names['PDIABTS1']: { 7:np.nan, 9:np.nan, 8:7},
                 column_names['PREDIAB2']: { 7:np.nan, 9:np.nan},
                 column_names['DIABTYPE']: { 7:np.nan, 9:np.nan},
                 column_names['INSULIN1']: { 7:np.nan, 9:np.nan},
                 column_names['CHKHEMO3']: { 88:0, 98:0, 77:np.nan, 99:np.nan},
                 column_names['EYEEXAM1']: { 7:np.nan, 8:0, 9:np.nan},
                 column_names['DIABEYE1']: {8:0, 7:np.nan, 9:np.nan},
                 column_names['DIABEDU1']: {8:0, 7:np.nan, 9:np.nan},
                 column_names['FEETSORE']: {2:0, 7:np.nan, 9:np.nan},
                }

# Class create_dataset

In [6]:
class create_dataset:
    def __init__(self, origin_dataset, columns_to_keep, column_values):
        self.data            = origin_dataset.copy()
        self.column_mapping  = columns_to_keep 
        self.value_mapping   = column_values
        self.non_int_columns = [self.column_mapping['HTM4'],
                                self.column_mapping['WTKG3'],
                                self.column_mapping['_BMI5']]
        
        
    def filter_interviews(self):
        self.data = self.data[self.data['DISPCODE']==1100]
        
    
    def filter_columns(self):
        selected_cols = list(self.column_mapping.keys())
        self.data = self.data[selected_cols]
        
    def convert_values(self):
        for col,mapping in self.value_mapping.items():
            self.data[col] = self.data[col].map(lambda x: mapping.get(x, x))
        
    def filter_nan(self):
        self.data.dropna(inplace=True)
        
    def rename_columns(self):
        self.data.rename(columns=self.column_mapping, inplace=True)
            
    def convert_data_types(self):
        for col in list( set(self.data.columns) - set(self.non_int_columns) ):
            self.data[col] = self.data[col].astype(int)
        for col in self.non_int_columns:
            self.data[col] /= 100
       
    def convert_nan_to_zero(self):
        for col in ['PDIABTS1','PREDIAB2','DIABTYPE','INSULIN1','CHKHEMO3','EYEEXAM1','DIABEYE1','DIABEDU1','FEETSORE']:
            self.data[self.column_mapping[col]].fillna(0, inplace=True)


In [7]:
def dataset_creation_pipeline(inputs, column_names, column_values):
    obj = create_dataset(inputs, column_names, column_values)
    obj.filter_interviews()
    obj.filter_columns()
    obj.rename_columns()
    obj.convert_nan_to_zero()
    obj.convert_values()
    obj.filter_nan()
    obj.convert_data_types()
    return obj

In [8]:
df = dataset_creation_pipeline(input_data, column_names, column_values)
df.data

,sex,age_group,education,has_insurance,income_group,race,marital_status,general_health,exercise_lately,sleep_time,...,diabetes,high_bp,prediabetes,diabetes_type,currently_insulin,Hemoglobin,dilated_pupils,eye_photo,diabetes_education,sore_feet
1,1,5,1,1,2,0,2,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,1,4,3,1,5,0,0,1,1,5,...,0,0,0,0,0,0,0,0,0,0
4,1,2,2,1,2,0,0,4,1,9,...,0,0,0,0,0,0,0,0,0,0
6,1,5,3,1,4,1,1,1,1,7,...,0,0,0,0,0,0,0,0,0,0
7,1,5,1,1,4,0,2,3,0,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445120,0,4,2,1,5,0,1,1,1,6,...,0,2,3,0,0,0,0,0,0,0
445124,0,5,3,1,5,6,0,3,1,7,...,2,0,0,2,2,1,0,2,0,0
445126,0,1,3,1,4,0,4,3,1,8,...,0,2,3,0,0,0,0,0,0,0
445128,1,3,3,1,4,1,0,0,1,7,...,0,1,3,0,0,0,0,0,0,0


In [9]:
df.data['diabetes'].value_counts()

diabetes
0    201672
2     35396
1      5767
Name: count, dtype: int64

In [10]:
df.data.to_csv(path+'diabetes_dataset.csv')

# TODO

TODO: 
- Find more descriptive and shorter column names
- Legend, mentioning what every column stands for
- upload to kaggle